<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Clustering-based" data-toc-modified-id="Clustering-based-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Clustering based</a></span><ul class="toc-item"><li><span><a href="#modeling" data-toc-modified-id="modeling-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>modeling</a></span></li></ul></li><li><span><a href="#LDA-based" data-toc-modified-id="LDA-based-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>LDA based</a></span><ul class="toc-item"><li><span><a href="#Kmeans-clustering-based-on-LDA-topic-distribution-representation" data-toc-modified-id="Kmeans-clustering-based-on-LDA-topic-distribution-representation-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Kmeans clustering based on LDA topic distribution representation</a></span></li></ul></li></ul></div>

# Clustering based
- Steps:
    1. Transform into TF-IDF matrix
    2. Dimension reduction into 200
    3. Clustering in cosine similarity space (since it is word)
    4. Assign labels with majority vote based on training set labels
    5. Prediction
        1. Transform test set into TF-IDF matrix
        2. Dimension reduction into 200
        3. Make prediction based on the clusters and mapping between clusters and labels from training set
    6. Evaluation
        1. Based on classification report

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Text-Classification/code")
!pip install pyLDAvis
!pip install gensim
!pip install pandas==1.3.0
import nltk
nltk.download('stopwords')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import numpy as np 
import pandas as pd
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline

from classification_utils import *
from clustering_utils import *
from eda_utils import *
from nn_utils_keras import *
from feature_engineering_utils import *
from data_utils import *
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


## modeling

In [3]:
train, test = load_data()
# train, upsampling_info = upsampling_train(train)

train_text, train_label = train_augmentation(train, select_comb=None)
test_text, test_label = test['text'], test['label']

# test_text = test_text.apply(lambda x: normal_string(x))
# train_text = train_text.apply(lambda x: normal_string(x))


may use cols: 
 ['global_index', 'doc_path', 'label', 'reply', 'reference_one', 'reference_two', 'tag_reply', 'tag_reference_one', 'tag_reference_two', 'Subject', 'From', 'Lines', 'Organization', 'contained_emails', 'long_string', 'text', 'error_message']


# LDA based 

In [4]:
num_topics = 100
lda, voc = fit_topic_model(train_text, num_topics=num_topics, save_name='gensim_lda_model')
# lda = load_gensim_LDA_model(save_name='lda_gensim_model')

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [5]:
# vis_lda = visualize_LDA_model(train_text, voc, lda)
# vis_lda

In [6]:
train_pred_group, train_topic_distribution = pred_topic_model(lda, train_text, vocabulary=voc)
group_to_label = link_group_to_label(train_label, train_pred_group)

test_pred_group, test_topic_distribution = pred_topic_model(lda, test_text, vocabulary=voc)
test_pred = test_pred_group.apply(lambda group: group_to_label[group])

Series([], dtype: float64)
Group to label mapping: 
Group 0 <-> label comp.graphics
Group 1 <-> label misc.forsale
Group 2 <-> label talk.politics.mideast
Group 3 <-> label comp.sys.ibm.pc.hardware
Group 4 <-> label rec.motorcycles
Group 5 <-> label comp.graphics
Group 6 <-> label sci.crypt
Group 7 <-> label rec.sport.baseball
Group 8 <-> label comp.graphics
Group 9 <-> label comp.sys.mac.hardware
Group 10 <-> label sci.space
Group 11 <-> label comp.os.ms-windows.misc
Group 12 <-> label rec.sport.baseball
Group 13 <-> label comp.sys.mac.hardware
Group 14 <-> label talk.politics.misc
Group 15 <-> label comp.sys.ibm.pc.hardware
Group 16 <-> label comp.sys.ibm.pc.hardware
Group 17 <-> label comp.os.ms-windows.misc
Group 18 <-> label sci.crypt
Group 19 <-> label rec.autos
Group 20 <-> label rec.sport.baseball
Group 21 <-> label comp.windows.x
Group 22 <-> label no_group
Group 23 <-> label rec.sport.baseball
Group 24 <-> label rec.autos
Group 25 <-> label misc.forsale
Group 26 <-> label mis

In [7]:
print(metrics.classification_report(y_true = test_label, y_pred=test_pred))

                          precision    recall  f1-score   support

             alt.atheism       0.11      0.04      0.06       319
           comp.graphics       0.10      0.01      0.01       389
 comp.os.ms-windows.misc       0.10      0.03      0.04       394
comp.sys.ibm.pc.hardware       0.20      0.05      0.07       392
   comp.sys.mac.hardware       0.26      0.32      0.29       385
          comp.windows.x       0.19      0.71      0.29       395
            misc.forsale       0.29      0.27      0.28       390
               rec.autos       0.06      0.01      0.02       395
         rec.motorcycles       0.20      0.05      0.08       398
      rec.sport.baseball       0.14      0.20      0.16       397
        rec.sport.hockey       0.20      0.00      0.00       827
               sci.crypt       0.13      0.36      0.19       396
         sci.electronics       0.00      0.00      0.00       393
                 sci.med       0.02      0.05      0.03       198
         

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Kmeans clustering based on LDA topic distribution representation

In [8]:
dtm_train, transform_mapper = dimension_reduction(train_topic_distribution, out_dim=2)
dtm_test = transform_mapper.transform(test_topic_distribution)

clusterer, clusters_to_labels = fit_clustering_model(dtm_train, train_label, num_clusters=2, metric='Cosine', repeats=10)
pred = pred_clustering_model(dtm_test, clusterer, clusters_to_labels)
print(metrics.classification_report(y_true = test_label, y_pred=pred))

Dimension reduction with truncate SVD:
   input columns with  100
   output columns with  2


/usr/local/lib/python3.7/dist-packages/scipy/linalg/decomp_qr.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  kwargs['lwork'] = ret[-2][0].real.astype(numpy.int)
/usr/local/lib/python3.7/dist-packages/scipy/linalg/decomp_qr.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, ch

Group to label mapping: 
Group 0 <-> label sci.med
Group 1 <-> label comp.sys.ibm.pc.hardware


                          precision    recall  f1-score   support

             alt.atheism       0.00      0.00      0.00       319
           comp.graphics       0.00      0.00      0.00       389
 comp.os.ms-windows.misc       0.00      0.00      0.00       394
comp.sys.ibm.pc.hardware       0.12      0.90      0.22       392
   comp.sys.mac.hardware       0.00      0.00      0.00       385
          comp.windows.x       0.00      0.00      0.00       395
            misc.forsale       0.00      0.00      0.00       390
               rec.autos       0.00      0.00      0.00       395
         rec.motorcycles       0.00      0.00      0.00       398
      rec.sport.baseball       0.00      0.00      0.00       397
        rec.sport.hockey       0.00      0.00      0.00       827
               sci.crypt       0.00      0.00      0.00       396
         sci.electronics       0.00      0.00

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
